In [1]:
import geograpy3
import spacy
import os
import spacy
from find_job_titles import FinderAcora
import pandas as pd
import re
from ipykernel import kernelapp as app

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/dalila.benachenhou/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/dalila.benachenhou/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /Users/dalila.benachenhou/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /Users/dalila.benachenhou/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dalila.benachenhou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/dalila.benachenhou/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is alrea

In [2]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
'''from nltk import RegexpParser
from nltk import Tree
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords'''

'from nltk import RegexpParser\nfrom nltk import Tree\nfrom nltk.tokenize import sent_tokenize, word_tokenize\nfrom nltk.corpus import stopwords'

In [3]:
#Load en_core_web_md augmented
nlp = spacy.load('./')

In [4]:
#Get Dictionary of Experience
#This can also be added to the dictionary spacy, and can be updated as needed
heading_dictionary = pd.read_csv("../Lower_big_headings_dictionary.csv")
heading_dictionary.drop_duplicates(inplace = True)
experience_headings = heading_dictionary[heading_dictionary.Label == "experience"]
skills_headings = heading_dictionary[heading_dictionary.Label == "skills"]
education_headings = heading_dictionary[heading_dictionary.Label == "education"]


In [5]:
import fitz
'''
read_pdf_resume: get the path and the name of the pdf resume and read all the pages into a string variable
Parameter:  pdf_document: a full path to a pdf resume
Return: text: all the text in all pages of the pdf document
'''
def read_pdf_resume(pdf_document):
    try:
        doc = fitz.open(pdf_document) 
        text = ''
        for i, page_n in enumerate(doc.pages()):
            page = page_n.getText("text")
            text += " "+page
    except :
        return ('')
    else:
        return(text)

In [6]:
'''
initial_cleaning:  split documents using return controler \n; remove empty lines
Parameter: doc_2_clean, a string variable containing the whole resume of a given person
'''

#is_symbol: check if the string has only at most 2 characters (so it may be a symbol)
def is_symbol(s):
    res = s.strip()
    l = list(s)
    if (len(l) < 3):
        return(True)
    else:
        return(False)
def remove_symbols(text_l):
    tmp = [x for x in text_l if not is_symbol(x)]
    return(tmp)
def initial_cleaning(doc_2_clean):
    text2 = doc_2_clean.split("\n")
    text2 = [i.strip() for i in text2]
    text2 = list(filter(lambda x: x != '', text2))
    text2 = remove_symbols(text2)
    return (text2)    

In [7]:
s = "Institute for Educational Sciences Fellow at University of Pennsylvania, Philadelphia, PA 2004-Present"
'''
get_date:  Extract dates from resume.  This is specifically for extracting the date in job experience 
( a range of 2 dates).  For education, one can just use Spicy to extract the date 
parameters:
s : a string
'''

def get_date(s):
    #Covers 90% of date patters
    date_pattern = "(\w{3,9}\.*\s+|[0-9]{2}\/)\d{4}\s*(\-|\–|To|to|TO)\s*((P|p|C|c)|(\w{3,9}\.*\s+|[0-9]{2}\/)\d{4})"
    date_pattern2 = "\d{4}\s*(\-|\–|To|to|TO)\s*(\d{4}|(P|p|C|c))"
    data_pattern3 ="\d{4}\-\d{2}\s*(\-|\–|To|to|TO)\s*((P|p|C|c)|\d{4}\-\d{2})"
    tp = re.search(date_pattern,s)
    if not tp:
        tp2 = re.search(date_pattern2,s)
        if not tp2:
            tp3 = re.search(data_pattern3,s)
            if tp3:
                start = tp3.span()[0]
                end  = tp3.span()[1]
                return [start,end, tp3.string[start:end]] 
            else:
                return [0,0,""]
        if tp2:
            start = tp2.span()[0]
            end  = tp2.span()[1]
            return [start,end, tp2.string[start:end]]
        
    else:
        print("pattern1")
        start = tp.span()[0]
        end  = tp.span()[1]
        return [start,end, tp.string[start:end]]

'''
get_date_and_remove_it_from_title:  find the date and remove it.
'''    
def get_date_and_remove_it_from_title(st):
    s = st.strip()
    start,end, date_s = get_date(s)
    print(s)
    if date_s:
        if start == 0:
            pat = r'(P|p|C|c)$'             
            match = re.search(pat, date_s)
            if match:
                print("found")
                end = end + 7
                s = s[end:]
            else:
                s = s[len(date_s):].strip()
        else:
            s = s[:start-1]
    else:
        date_s = ""
    return date_s,s

In [9]:
import re
'''
clean_title_df: initial cleaning of the titles.  Some titles will have tabs instead of space, or some non-digit 
characters
parameters:
s: string
'''
def clean_title_df(s):
    pattern1 = "\|\s|(\:)$"
    pattern2 = "^[a-z]\."
    s = s.replace("\t"," ")
    s = s.replace(u'\xa0', u' ')
    s = re.sub(pattern2,"",s)
    s =  re.sub(pattern1, "", s)
    s = re.sub("^[:xdigit:]","", s)
    s = s.strip()
    return(s) 



In [10]:
'''
gather_headings:  get the headings name, and their location for a given resume
Parameters:  doc_list: resume as a list of strings; resume_name: the resume file name
Return: blocks_df: a pandas dataframe with the resume name, position of a heading, and its name
tmp check_if_headings(block)
'''
def check_if_potential_title(s):
    s = s.strip()
    start,end, date_s =  get_date(s)
    if date_s:
        return(True)
    if re.search("^[0-9]{4}\Z",s):
        return(True)
    pattern = "^[A-Z]\s[A-Z]\s[A-Z]\s"
    s = s.replace("\t"," ")
    s = s.replace("&","")
    s = s.replace("|","")
    if re.search(pattern,s):
            return(True)
    else:
        tmp = s.split(" ")
        tmp = [x for x in tmp if not x == '']
        tmp = [x.strip() for x in tmp]
        if (len(tmp) == 1) and tmp[0][0].isupper():
            if re.search("[a-z]+\.$",tmp[0]):
                return(False)
            else:
                return(True)
        
        if (len(tmp) == 2) and (tmp[0][0].isupper()):
             return(True)
        elif (len(tmp) >= 3):
            tmp = [x for x in tmp if x not in ["of","and","for","in","at"]]
            if tmp[0].isupper():
                return(True)
            if tmp[0][0].isupper():
                if tmp[1][0].isupper():
                    return(True)
                else:
                    return(False)
            else:
                return(False)
        else:
            return(False)


In [12]:
def multiple_replace(string, reps, re_flags = 0):
    """ Transforms string, replacing keys from re_str_dict with values.
    reps: dictionary, or list of key-value pairs (to enforce ordering;
          earlier items have higher priority).
          Keys are used as regular expressions.
    re_flags: interpretation of regular expressions, such as re.DOTALL
    """
    if isinstance(reps, dict):
        reps = reps.items()
    pattern = re.compile("|".join("(?P<_%d>%s)" % (i, re_str[0])
                                  for i, re_str in enumerate(reps)),
                         re_flags)
    return pattern.sub(lambda x: reps[int(x.lastgroup[1:])][1], string) 

'''
check_number_uppercases_words:  check if the words in a line are upper case.  However, it makes sure to 
remove digits, some stopwords, that are lower cased in a title.  This function is used after the first pass of 
extracting titles from a resume.  
Parameters:
s: string
Return:  True if a true title, else it returns false'''

def check_number_uppercases_words(s):
    replacements = [("\t", ""), ("&", ""),("\/" , ""),(":",""),(",",""),("-"," "), (" for ", " "),
                    ("–" , ""),(" at "," "),(" of "," "),(" in "," "),(" to "," "),(" and "," "),("[0-9]","")]
    s = multiple_replace(s, replacements)
    tmp = s.split(" ")
    tmp = [x.strip() for x in tmp if x != " "]
    tmp = [x for x in tmp if x]
    upper_val = [x for x in tmp if  x[0].isupper()]
    if (len(tmp) <= 2 & len(upper_val) == 1):
        return(True)
    elif len(tmp) == len(upper_val):
        return(True)
    else:
        return(False) 

'''
gather_headings:  get all the headings (titles) from the resume.
Paramerters:
  doc_list:  reading pdf to text result in a text with return lines, so a split result in a list of strings
  resume_name: the file name of the resume
  
Returns:
  blocks_df: a pandas df
'''   
def gather_headings(doc_list, resume_name):
    blocks_df = pd.DataFrame()
    for i,block in enumerate(doc_list):
        block = clean_title_df(block)
        tmp = block.split()
        if len(tmp) == 1:
            block = block.strip()
            if block[0].isupper():
                tmp_df = pd.DataFrame({"Resume_Name": resume_name,"Block_Pos" : i, "Block_Title" : block},index = [i])
                blocks_df = blocks_df.append(tmp_df)
        elif check_if_potential_title(block):
            tmp_df = pd.DataFrame({"Resume_Name": resume_name,"Block_Pos" : i, "Block_Title" : block},index = [i])
            blocks_df = blocks_df.append(tmp_df)
    blocks_df.reset_index()
    tp = [ x for ix, x in blocks_df.iterrows() if check_number_uppercases_words(x.Block_Title)]
    blocks_df = pd.DataFrame(tp)
    #blocks_df.reset_index()
    return(blocks_df)   

In [13]:
def V2_gather_headings(doc_list, resume_name):
    blocks_df = pd.DataFrame()
    pattern2ignore = "\A([A-Z][a-z]{3,}\s*){2,3}\."
    for i,block in enumerate(doc_list):
        block = clean_title_df(block)
        tmp = block.split()
        if check_if_potential_title(block) and not re.search(pattern2ignore,block):
            tmp_df = pd.DataFrame({"Resume_Name": resume_name,"Block_Pos" : i, "Block_Title" : block},index = [i])
            blocks_df = blocks_df.append(tmp_df) 
    #blocks_df.reset_index()
    tp = [ x for ix, x in blocks_df.iterrows() if check_number_uppercases_words(x.Block_Title)]
    blocks_df = pd.DataFrame(tp)
    #blocks_df.reset_index()
    return(blocks_df)   


In [14]:
# Create an nlp object
from spacy.matcher import Matcher
def maybe_company(s):
    m = s.split()
    if m[0].isupper():
        return(True)
    else:
        return(False)
    
def get_date_or_company(s):
    date_s = ""
    company_s = ""
    doc = nlp(s)
    
    for ent in doc.ents:
        if ent.label_ == "DATE":
            date_s = ent.text
            if not date_s:
                _,end,date_s = get_date(s)  
        elif ent.label_ == "ORG":
            company_s = ent.text
    return(date_s,company_s)
def extract_degree_level(s):
    doc = nlp(s)
    degree = ""
    degree_type = ""
    for ent in doc.ents:
        if ent.label_ == "EDU":
            degree_type = ent.ent_id_
            degree = ent.text
            return degree, degree_type
    return degree, degree_type 
def extract_all_info_degree(s):
    degree_dict = {"GPE":"","ORG":"","Bachelor":"","Master":"","Doctor":"","Major":"","DATE":""}
    degree_dict["DATE"], s = get_date_and_remove_it_from_title(s)
    s = s.replace("Magna cum Laude","")
    s = s.replace("cum Laude","")
    if re.search("\s(A|a)t\s",s):
        tp = re.split("\s+(A|a)t\s+",s)
        l = len(tp)-1
        degree_dict["ORG"] = tp[l]
        s= tp[0]
    doc = nlp(s)
    degree_c = 1
    for ent in doc.ents:
        if ent.label_ == "DATE":
            if not degree_dict["DATE"]:
                degree_dict[ent.label_] = ent.text
                s = s.replace(ent.text,"")
            
        else:
            if ent.label_ == "EDU":
                degree_dict[ent.ent_id_] = ent.text
                s = s.replace(ent.text,"")
            elif ent.label_ == "ORG":
                if not degree_dict["ORG"]:
                        degree_dict["ORG"] = degree_dict["ORG"]+" | "+ent.text
                        s = s.replace(ent.text,"")
            elif ent.label_ == "GPE":
                if not degree_dict["GPE"]:
                        degree_dict["GPE"] = ent.text
                        s = s.replace(ent.text,"")
            
            else:
                if ent.label_ in list(degree_dict.keys()):
                        degree_dict[ent.label_] = ent.text
                        s = s.replace(ent.text,"")
    if not degree_dict["DATE"]:
        tp = re.search("\s\d{4}$",s)
        if tp:
            degree_dict["DATE"]= s[tp.start():]
            s = s[:tp.start()]
           
            
    if not degree_dict["GPE"]:
        degree_dict["GPE"]= extract_city(s)
        if degree_dict["GPE"]:
            l = len(degree_dict["GPE"])
            degree_dict["GPE"] = degree_dict["GPE"][l-1]
            s = s.replace(degree_dict["GPE"],"")
        else:
            degree_dict["GPE"]= ""
    if not degree_dict["Major"]:
        s = s.replace(".","")
        s = re.sub("\,","",s)
        s = s.strip()
        if re.search("School|University|College|Institute",s):
            degree_dict["Major"]= degree_dict["ORG"]
            degree_dict["ORG"] = re.sub("[^a-zA-Z&\s]","",s).strip()
        else:  
            tp = re.split("(IN|in|In)",s)
            if len(tp) > 1:
                degree_dict["Major"]= re.sub("[^[a-zA-Z&\s]","",tp[len(tp)-1]).strip()
            else:
                degree_dict["Major"]= s
        
    return degree_dict

In [783]:
#get_date_or_company("South Dakota State University, Brookings MA, Economics 2014")
s = "WESTERN KENTUCKY UNIVERSITY, Bowling Green, KY | Bachelor of Arts – Economics & Political Science"
date_s,company_s = get_date_or_company(s)
if not date_s:
    date_s, s = get_date_and_remove_it_from_title(s)
    if not date_s:
        tp = re.search("\s+\d{4}$",s)
        if tp:
            date_s = s[tp.start():]
            s = s[:tp.start()]
    else:
        s = re.replace(date_s,s)
else:
    s = s.replace(date_s," ")
if company_s:
    s = s.replace(company_s," ")
s = s.replace(",","") 
s = s.strip()
city_s = extract_city(s)
if len(city_s)>= 1:
    city_s = city_s[0]
if city_s:
    s = s.replace(city_s,"")
degree, degree_type = extract_degree_level(s)
if degree:
    s = s.replace(degree,"")
    s = s.strip()
    major = s
else:
    s = s.strip()
    major = s
    

            


WESTERN KENTUCKY UNIVERSITY, Bowling Green, KY | Bachelor of Arts – Economics & Political Science


In [15]:
'''def get_degree_university_date(s):
doc = nlp("South Dakota State University, Brookings MA, Economics 2014")
for ent in doc.ents:
    print(ent.label_,ent.text)'''
#city_s = extract_city("Augustana University, Sioux Falls BA, Econ & Math 2012")
#print(major, " * ",city_s," * ",degree, " * ", company_s, " * ",date_s)
#s = "Fordham University, New York City PhD, Economics In progress, ABD"
#extract_city("New York City")
def get_job_info(s):
    found = False
    found_date = False
    job_dict = {"DATE":"","ORG":"","GPE":"","Job_Title":""}
    job_dict["GPE"] = extract_city(s)
    if job_dict["GPE"]:
        l = len(job_dict["GPE"])-1
        job_dict["GPE"] = job_dict["GPE"][l]
        s = s.replace(job_dict["GPE"],"")
    else:
        job_dict["GPE"] = ""
    doc = nlp(s)
    n = 1
    for ent in doc.ents:
        '''if not found:
            start,end, job_title = list(get_job_title(ent.text))[0]
            if job_title:
                job_dict["Job_Title"] = job_title
                found = True'''
        print("s ",s)
        if ent.label_ == "GPE":
            if not job_dict["GPE"]:
                job_dict["GPE"] = ent.text
                s = s.replace(ent.text,"")
        elif ent.label_ == "DATE":
            if n == 1:
                job_dict["DATE"] =  ent.text
                s = s.replace(ent.text,"")
                n += 1
            else:
                job_dict["DATE"] = job_dict["DATE"]+" - "+ent.text
                s = s.replace(ent.text,"")
            found_date = True
        elif ent.label_ == "ORG":
            job_dict[ent.label_] = job_dict[ent.label_]+" : "+ent.text
            s = s.replace(ent.text,"")
            
    if not job_dict["DATE"]:    
        job_dict["DATE"],s = get_date_and_remove_it_from_title(s)
    
    if not found:
        s = s.replace(".","")
        s = re.sub("[A-Z]{2}","",s)
        s = re.sub("[^a-zA-Z\s]","",s)
        job_dict["Job_Title"] = s.strip()
    if isinstance(job_dict["GPE"],list):
        job_dict["GPE"] = ""
    return job_dict

In [16]:
def extract_city(s):
    places = geograpy3.get_place_context(text = s)
    return (places.cities)

In [17]:
def count_n_POS(s):
    tmp = s.split(",")
    tp = nltk.pos_tag(nltk.word_tokenize(tmp[0]))
    res =[val for i, val in tp]
    res = set(res) 
    return(res)
def is_phone_number(s):
    res = re.sub(r'[^\w^\s]', '', s) 
    res = res.replace(' ',"")
    l = list(res)
    if res.isnumeric() and (len(l) >= 9):
        return(True)
    else:
        return(False)

In [18]:
#"from find_job_titles import FinderAcora"

#s = 'SR. DATA PROCESSING SYSTEMS ANALYST'
#Find Job Title
def get_job_title(s):
    f = re.search("(QA\s|ASSISTANT|Assistant|ANALYST|Analyst|PRINCIPAL|Principal)",s)
    if f:
        start, end = f.span(0)
        title_s = f.string
        print(start, end)
        yield start, end, s
    else:
        s = s.title()
        finder=FinderAcora()
        result = finder.finditer(s)
        try:
            gen = iter(result)
            it = next(gen)
            start = it.start
            end   = it.end
            title_s = it.match
            yield start, end, title_s
        except RuntimeError:
            #return start, end, title_s
            yield 0,0,""

'''s = "Five Corner Strategies, Senior Project Manager / Data Director 02/2018-03/2019 Washington, DC"
s= "Research Assistant, Fordham University Econometrics 2014– Present"
s = "Humana (Stars Data Analytics), Louisville, Ky. — Data Science Architect"
s = "GRADUATE COURSE ASSISTANT"
a,b,c = get_job_title(s)
print(a, b, c )'''
a, b, c = list(get_job_title("Director of Client Success"))[0]
print(a, b, c )

0 8 Director


In [19]:
# Teach For America – High School STEM Teacher, Chicago, IL May 2011-December 2012
#Or BOSTON POLICE DEPARTMENT – Boston, MA 2001 TO 2007
'''Pattern if - exist.  In this case, we can have 2 possibilities, either 
the first value is the company or the second is the company'''
s = "2013-2017 Founder & President - Boston Quantitative Innovations Pennington, NJ"
'''If date ad city found than we can assume that we have the company ad the job title
'''
def get_city_and_remove_it(st):
    city_s = extract_city(st)
    if city_s:
        city_s = city_s[len(city_s)-1]
        matches = re.finditer(city_s, st)
        matches_positions = [match.start() for match in matches]
        matches_positions = matches_positions[len(matches_positions)-1]
        s = st
        s = s[:matches_positions-1]
        print("Company ",s)
        return city_s, s
    else:
        return "",st

In [20]:
#In here we assume city and company after job description
#but we have Company, City, KY. - Title

'''
CHECK IT OUT 
'''
def get_title_company_AT(st,pattern_at):
    tp = re.split(pattern_at,st)
    tp = [x.strip() for x in tp if x]
    tp = [x for x in tp if x not in ["at","At"]]
    job_title = tp[0]
    city_s = extract_city(tp[1])
    if city_s and re.search("\,",tp[1]):
        tp_city = re.split("\,",tp[1])
        print(tp_city)
        company_s = tp_city[0]
    return job_title, company_s

def get_title_company_commma(st):
    tp = re.split(punct,s)
    tp = [x.strip() for x in tp if x]
    tp = [x for x in tp if x not in [",","|"]]
    start, end, job_title = list(get_job_title(tp[0]))[0]
    if end == 0:
        job_title = tp[1]
        company_s = tp[0]
    else:
        job_title = tp[0]
        company_s = tp[1]
    return job_title, company_s

def get_work_heading(st,punct):
    s = st.strip()
    date_s, s = get_date_and_remove_it_from_title(s)
    pattern_at = "\s+(at|At|AT)\s+"
    pattern_comma = "\s*(\,|\||/)\s+"
    pattern_under = "\s*(-|—\|)\s*"
    if re.search(pattern_at,s):
        job_title, company_s = get_title_company_AT(s,pattern_at)
    elif re.search(pattern_under,s):    
        tp = re.split(punct,s)
        tp = [x.strip() for x in tp if x]
        tp = [x for x in tp if x not in ["-","—","|"]]
        print(tp)
        #print(len(tp),"tp " ,tp)
        if len(tp)== 3 :
            tp[1] = tp[len(tp)-1]
        city_s, company_s = get_city_and_remove_it(tp[1])
        if not city_s:
            city_s, company_s = get_city_and_remove_it(tp[0])
            job_title = tp[1]
        if  city_s: 
            job_title = tp[0]
          
                    
    else :
        tp = re.split(pattern_comma,s)
        print(tp)
        tp = [x.strip() for x in tp if x]
        tp = [x for x in tp if x not in [",","|","/"]]
        start, end, job_title = list(get_job_title(tp[0]))[0]
        if end == 0:
            job_title = tp[1]
            company_s = tp[0]
        else:
            job_title = tp[0]
            company_s = tp[1]
    job_dict = {"job Title" : job_title, "Company" : company_s,"Dates" : date_s }
    return(job_dict)
'''#s = "2004-2008 Institute for Educational Sciences Fellow at University of Pennsylvania, Philadelphia, PA"
s = "2011-2013 Director of Research & Evaluation at Camden Coalition of Healthcare Providers, Camden, NJ"
s = "UNIVERSITY OF COLORADO BOULDER at GRADUATE COURSE ASSISTANT"
pattern1 = "(\s+(at|At)\s+|\s*(\–|\-|\—|\|)\s*)"
pattern2 = " at | At "
get_work_heading(s,pattern1)'''
#tp = re.split("\s*(–|-)\s*",s)
#print(re.split("\sat\s","this at is"))
#city_s = extract_city(s)

pattern1 = "(\s+(at|At)\s+|\s*(\–|\-|\—|\|)\s*)"
s = "Assistant-Google, Chicago, IL"
get_work_heading(s,pattern1)
#places = geograpy3.get_place_context(text = s)
#re.split("\s*(\,|\||/)\s+",s)

Assistant-Google, Chicago, IL
['Assistant', 'Google, Chicago, IL']
Company  Google,


{'job Title': 'Assistant', 'Company': 'Google,', 'Dates': ''}

In [195]:
tp = re.split(",|\|","This is a function with no ")
print(tp)

['This is a function with no ']


In [21]:
s = "NRCC (National Republican Congressional Committee), Director of Analytics 03/2016-12/2017 Washington, DC"
s2 = "Director of Analytics at NRCC (National Republican Congressional Committee),  Washington, DC 03/2016-12/2017"

'''RETHINK THIS'''
def get_Title_Co_Commas(st):
    job_title = ""
    company_s = ""
    date_s = ""
    s = st.strip()
    date_s, s = get_date_and_remove_it_from_title(s)
    print(date_s)
    matches = re.finditer("(\s+(a|A)t\s+|\s*(\—|\-)\s*)", s)
    matches_positions = [match.start() for match in matches]
    if matches_positions:
        pattern1 = "(\s+(at|At)\s+|\s*(\–|\-|\—|\|)\s*)"
        res = get_work_heading(s,pattern1)
        job_title = res["job Title"]
        company_s = res["Company"]
    else:
        matches = re.finditer(",|\|", s)
        matches_positions = [match.start() for match in matches]
       
        if matches_positions:
            tp = re.split("(,|\|)",s) 
            tp = [x for x in tp if x not in [",","|"]]
            print(" SSSS ",s, len(tp))
            print(" TPP ",tp)
            if len(tp) == 2:
                _,company_s = get_date_or_company(tp[0])
                if company_s:
                    start,end,job_title = list(get_job_title(tp[1]))[0]
                else:
                    start,end,job_title = list(get_job_title(tp[0]))[0]
                    _, company_s = get_city_and_remove_it(tp[1])
                        
            
                
        else:
            start,end,job_title = list(get_job_title(s))[0]
            if end == 0:
                company_s = s
                
    job_dict = {"job Title" : job_title, "Company" : company_s,"Dates" : date_s }
    return job_dict
#s = "Humana (Stars Data Analytics), Louisville, Ky. — Data Science Architect"
#print(get_Title_Co_Commas("Teacher , Google"))
print(get_Title_Co_Commas("Assistant - Google, Chicago, IL"))
#print(get_work_heading(s,pattern1))
get_date_or_company("Assistant")

Assistant - Google, Chicago, IL

Assistant - Google, Chicago, IL
['Assistant', 'Google, Chicago, IL']
Company  Google,
{'job Title': 'Assistant', 'Company': 'Google,', 'Dates': ''}


('', '')

In [22]:
def is_place(s):

    if extract_city(s) == []:
        return (False)
    else:
        return (True)
def is_email(s):
    pat = r'.*?@(.*)\..*'  
    pat = r'[\w.-]+@[\w.-]+'
    match = re.search(pat, s)
    if not match:
        return (False)
    else:
        return(True)
    
def is_github_or_linkedIn(s):
    pattern2 = '(g|G)ithub|(l|L)inked(i|I)n'
    if  any(re.findall(pattern2, s, re.IGNORECASE)):
        return(True)
    else:
        return(False)

In [23]:
def find_heading_position(resume_df, chosen_heading):
    tmp_l = [x.strip() for x in chosen_heading]
    for i, item in enumerate(resume_df.Block_Title):
        if str(item).lower() in tmp_l:
            return (int(i))
    return(-1)

In [24]:
def V2_get_start_and_end_experience(resume_df,experience_headings):
    global heading_dictionary
    experience_headings = heading_dictionary[heading_dictionary.Label == "experience"]
    skills_headings = heading_dictionary[heading_dictionary.Label == "skills"]
    education_headings = heading_dictionary[heading_dictionary.Label == "education"]
    resume_df = resume_df.reset_index()
    pos = find_heading_position(resume_df, experience_headings.Block_Title)
    
    if (pos == -1):
        print("Please add the name of the Experience heading to big_heading_dictionary")
        return (0,0)
    else:
        start_t = resume_df.Block_Pos.iloc[pos]+1
        pos = find_heading_position(resume_df, education_headings.Block_Title)
        
        if (pos == -1):
            print("Please add the heading to Education to the big_headings_dictionary.csv")
            education_pos = -1 
        else:
            education_pos = resume_df.Block_Pos.iloc[pos]+1
        pos = find_heading_position(resume_df, skills_headings.Block_Title)
        
        if (pos == - 1):
            skills_pos = -1
        else:
            skills_pos = resume_df.Block_Pos.iloc[pos]+1
                
        if ( start_t < education_pos) & (start_t < skills_pos):
            end_extract_pos = min(education_pos,skills_pos)
        elif ( start_t > education_pos) &(start_t < skills_pos):
            end_extract_pos = skills_pos
        elif ( start_t < education_pos) & (start_t > skills_pos):
            end_extract_pos = education_pos
        else: 
            end_extract_pos = len(resume_l)
        return(start_t,end_extract_pos)

In [25]:
#Detect start of an experience (Company name, location, position, place)

def V2_detect_start_experience(text_l,pos,n):
    global keep_Experience_headings
    s = text_l[pos]
    s = s.replace("\t",' ')
    i = pos
    not_start = True
    while not_start:
        _,_,date_s = get_date(s)
        if (not is_symbol(s) and (len(count_n_POS(s)) < 4) and i < n) or check_if_potential_title(s):
            if (i >= len(text_l)) :
                not_start = False
            else:
                keep_Experience_headings =keep_Experience_headings.append(pd.DataFrame({"Exp_Heading" : s}, index=[0]), ignore_index = True)
                s = text_l[i]
                s = s.replace("\t",' ')
                
            i += 1
        elif(i >= n):
            not_start = False
        elif (len(count_n_POS(s)) >= 4):
            not_start = False
        
        
    return (i)

In [26]:
def V3_detect_start_experience(text_l,pos,n):
    s = text_l[pos]
    pos_l = pos
    start_pos =  next(iter(resume_df[resume_df['Block_Pos']== pos_l-1].index), -1)
    while start_pos:
        pos_l += 1
        start_pos = next(iter(resume_df[resume_df['Block_Pos']== pos_l-1].index), -1)
        if pos_l >= n:
            start_pos = False
    return(pos_l-1) 

In [343]:
print(start_t)
end = V3_detect_start_experience(resume_l, start_t, start_t+10)
resume_l[start_t:end]

10


['Software Engineer',
 'HeavyWater Inc -  Philadelphia, PA - 2017-03 - Present',
 '• Responsible for automation of services provided through AWS (Lambda, EC2, SWF) in Document Processing',
 'Virtual Assistant.',
 '• Working on DevOps principles to improve system performance at different levels in existing infrastructure.',
 '• Responsible for operations and monitoring of production system.',
 '• Working on improvement of existing machine learning algorithms to extract data points accurately.',
 'Machine Learning Engineer',
 'Lumidatum Inc - 2016-11 - 2017-02']

In [27]:
#Extract the paragraph or block with the experience 

def extract_experience(text_l,pos,n):
    
    i = pos
    not_end = True
    experience_txt = ""
    #Is lower, and number of words is more than 
    while not_end:
        if i >= n:
            not_end = False
        else:
            s = text_l[i]
            s = s.replace("\t",' ')
            if check_if_potential_title(s) :
                not_end = False
            else: 
                if is_github_or_linkedIn(s) or is_email(s) or is_phone_number(s):
                    i = n
                    not_end = False
                else:
                    experience_txt = experience_txt + "\n"+ s
            i += 1
    return (experience_txt, i-1) 

In [28]:
def get_all_experiences(resume_l,start_t,end_extract_pos):

    experience_df = pd.DataFrame()
    keep_extracting_exp = True
    len_resume = end_extract_pos
    start_pos = start_t
    counter = 1
    while keep_extracting_exp:
        start_exp = V2_detect_start_experience(resume_l,start_pos,len_resume)
        if start_exp == len_resume - 1:
            keep_extracting_exp = False
        #break
        else:
            exp_txt,end_exp = extract_experience(resume_l,start_exp-1,len_resume)
            start_pos = end_exp
            experience_df = experience_df.append({"Experiences" : exp_txt,"Counter": counter},ignore_index = True)
            counter += 1
            if end_exp >= len_resume - 1:
                keep_extracting_exp = False
            
    return(experience_df)

In [ ]:
def tag_headings(resume_df):
    for index, row in resume_df.iterrows():
        degree,degree_type = extract_degree_level(row["Block_Title"])
        if degree:
            
            degree_info = extract_all_info_degree(row["Block_Title"])
            #
            #    print(key, " * ",index)
            for key, val in degree_info.items():
                resume_df.loc[index,key] = val
        else:
            job_info = get_job_info(row["Block_Title"])
            print(" JOB INFO ", job_info)
            for key, val in job_info.items():
                resume_df.loc[index,key] = val
            #tp = [1 for x in list(job_info.values()) if x]
            #if tp:
             #   for key, val in job_info.items():
             #       print(key, " * ",index)
             #       resume_df.iloc[index,key] = val
    return resume_df

In [1008]:
path_name = "../CFDS/"
resume_name = "82143925.pdf"

keep_Experience_headings = pd.DataFrame()
pdf_document = path_name+ resume_name 
text = read_pdf_resume(pdf_document)
resume_l = initial_cleaning(text)
resume_df = V2_gather_headings(resume_l,resume_name)
#start_t, end_extract_pos = V2_get_start_and_end_experience(resume_df,experience_headings)
#resume_df.reset_index(inplace=True,drop=True)
#resume_df2 = tag_headings(resume_df)
#resume_df2.loc[resume_df2.Block_Pos < start_t, "Job_Title"] = ""

#total_experience = get_all_experiences(resume_l,start_t,end_extract_pos)
#resume_df

pattern1
pattern1
pattern1
pattern1
pattern1
Please add the heading to Education to the big_headings_dictionary.csv


In [1011]:
resumes2process = pd.read_csv("../CFDS_Resume_2_Process.csv")

In [1014]:
os.getcwd()

'/Users/dalila.benachenhou/Documents/HSS/HSS_Resumes'

In [1017]:
all_job_titles = pd.DataFrame()
path_name = "../CFDS/"
resumes2process = pd.read_csv("../CFDS_Resume_2_Process.csv")
pdf_files = resumes2process.CFDS_Resume
for resume_name in pdf_files:
    keep_Experience_headings = pd.DataFrame()
    pdf_document = path_name+ resume_name 
    text = read_pdf_resume(pdf_document)
    if text:
        resume_l = initial_cleaning(text)
        resume_df = V2_gather_headings(resume_l,resume_name)
        if resume_df.empty:
            print("No Headings")
        else:
            start_t, end_extract_pos = V2_get_start_and_end_experience(resume_df,experience_headings)
            resume_df.reset_index(inplace=True,drop=True)
            resume_df2 = tag_headings(resume_df)
            resume_df2.loc[resume_df2.Block_Pos < start_t, "Job_Title"] = ""
            all_job_titles = all_job_titles.append(resume_df2,ignore_index = True)
    

pattern1
pattern1
pattern1
pattern1
s  JIALU HUANG
HUANG
 JOB INFO  {'DATE': '', 'ORG': ' : JIALU', 'GPE': '', 'Job_Title': 'G'}
EDUCATION
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'N'}
s   College London, United Kingdom.
s   College London, United Kingdom.
College London, United Kingdom.
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': 'Imperial', 'Job_Title': 'College London United Kingdom'}
s  September 2018 - September 2019
 JOB INFO  {'DATE': 'September 2018 - September 2019', 'ORG': '', 'GPE': '', 'Job_Title': ''}
MSc Statistics
 DEGREE INFO  {'GPE': '', 'ORG': '', 'Bachelor': '', 'Master': 'MSc', 'Doctor': '', 'Major': 'Statistics', 'DATE': ''}
s  The  of Edinburgh, United Kingdom.
s  The  of Edinburgh, United Kingdom.
The  of Edinburgh, United Kingdom.
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': 'University', 'Job_Title': 'The  of Edinburgh United Kingdom'}
s  September 2016-July 2018
 JOB INFO  {'DATE': 'September 2016-July 2018', 'ORG': '', 'GPE': '', 'Job_Title': ''}

s  PCA in Coursera
s   in Coursera
in
 JOB INFO  {'DATE': '', 'ORG': ' : PCA : Coursera', 'GPE': '', 'Job_Title': 'in'}
s  Fundamentals of Digital Marketing
Fundamentals of
 JOB INFO  {'DATE': '', 'ORG': ' : Digital Marketing', 'GPE': '', 'Job_Title': 'Fundamentals of'}
s  GOOGLE Online Course,Digital Active
s   Online Course,Digital Active
Online Course,
 JOB INFO  {'DATE': '', 'ORG': ' : GOOGLE : Digital Active', 'GPE': '', 'Job_Title': 'Online Course'}
TECHNICAL SKILLS
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'L'}
s  Java

 JOB INFO  {'DATE': '', 'ORG': ' : Java', 'GPE': '', 'Job_Title': ''}
Tensorﬂow
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Tensorow'}
Learning
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Learning'}
Project Management
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Project Management'}
s  Digital Marketing

 JOB INFO  {'DATE': '', 'ORG': ' : Digital Marketing', 'GPE': '', 'Job_Title': ''}
Transmission,3

s  MVC Framework

 JOB INFO  {'DATE': '', 'ORG': ' : MVC Framework', 'GPE': '', 'Job_Title': ''}
ALM, Artifactory, Confluence, MS Visio and Teams,
 DEGREE INFO  {'GPE': 'Artifactory', 'ORG': ' | ALM', 'Bachelor': '', 'Master': 'MS', 'Doctor': '', 'Major': 'Confluence  Visio and Teams', 'DATE': ''}
MS Office Suite, Slack, ServiceNow and Splunk
 DEGREE INFO  {'GPE': '', 'ORG': ' | ServiceNow', 'Bachelor': '', 'Master': 'MS', 'Doctor': '', 'Major': 'Office Suite Slack  and Splunk', 'DATE': ''}
ACCOMPLISHMENTS
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'S'}
s  Wen-Tsong Shiue
Wen-Tsong Shiue
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'WenTsong Shiue'}
s  Director of Engineering - Data Scientist - Uniqlinc Inc
Director of
 JOB INFO  {'DATE': '', 'ORG': ' : Engineering - Data Scientist - Uniqlinc Inc', 'GPE': '', 'Job_Title': 'Director of'}
s  , TX
,
 JOB INFO  {'DATE': '', 'ORG': ' : TX', 'GPE': 'Austin', 'Job_Title': ''}
Key Accomplishment
 JOB INFO  {'DATE'

Area
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Area'}
s  Memory Data Modeling and Exploration, Research Associate
, Research Associate
 JOB INFO  {'DATE': '', 'ORG': ' : Memory Data Modeling and Exploration', 'GPE': '', 'Job_Title': 'Research Associate'}
s  Arizona  - 1996 - 2000
Arizona  - 1996 - 2000
 JOB INFO  {'DATE': '1996 - 2000', 'ORG': '', 'GPE': 'State University', 'Job_Title': 'Arizona'}

 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': 'Research', 'Job_Title': ''}
Area
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Area'}
Adjunct Professor
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Adjunct Professor'}
TamKung  - 1994 - 1996
 JOB INFO  {'DATE': '1994 - 1996', 'ORG': '', 'GPE': 'University', 'Job_Title': 'TamKung'}
EDUCATION
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'N'}
s  PhD. in Electrical Engineering and Computer Science
PhD. in
 JOB INFO  {'DATE': '', 'ORG': ' : Electrical Engineering and Computer Science', 'GPE': 

s  Motorola, 
,
 JOB INFO  {'DATE': '', 'ORG': ' : Motorola', 'GPE': 'Bangalore', 'Job_Title': ''}
s  1997 – 1999
 JOB INFO  {'DATE': '1997 – 1999', 'ORG': '', 'GPE': '', 'Job_Title': ''}
s   & Toubro, Mumbai
& Toubro, Mumbai
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': 'Larsen', 'Job_Title': 'Toubro Mumbai'}
Graduate Engineer - Trainee
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Graduate Engineer  Trainee'}
s  1996 – 1997
 JOB INFO  {'DATE': '1996 – 1997', 'ORG': '', 'GPE': '', 'Job_Title': ''}
Education
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Education'}
s  2002 – 2004
s   – 2004
 JOB INFO  {'DATE': '2002 - 2004', 'ORG': '', 'GPE': '', 'Job_Title': ''}
s  Duke , Fuqua School of Business, Durham, NC, USA
s   , Fuqua School of Business, Durham, NC, USA
s   , , Durham, NC, USA
s   , , Durham, NC, USA
, , Durham, NC, USA
 JOB INFO  {'DATE': '', 'ORG': ' : Duke : Fuqua School of Business', 'GPE': 'University', 'Job_Title': 'Durham  A'}
s  1992 – 1996
 JOB 

...

mupdf: cannot open ../CFDS/111384356.pdf: No such file or directory


Meshry
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Meshry'}
s  El Mechry, El Koudouss
s  , El Koudouss
, El Koudouss
 JOB INFO  {'DATE': '', 'ORG': ' : El Mechry', 'GPE': '', 'Job_Title': 'El Koudouss'}
EXPERIENCE
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': ''}
s  Summer 2019
 JOB INFO  {'DATE': 'Summer 2019', 'ORG': '', 'GPE': '', 'Job_Title': ''}
Econometrics
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Econometrics'}
s  2014– Present
 JOB INFO  {'DATE': '2014', 'ORG': '', 'GPE': '', 'Job_Title': 'Present'}
Statistics & Econ
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Statistics  Econ'}
s  2014 – Present
 JOB INFO  {'DATE': '2014', 'ORG': '', 'GPE': '', 'Job_Title': 'Present'}
Microeconomics
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Microeconomics'}
s  2012-2014
 JOB INFO  {'DATE': '2012-2014', 'ORG': '', 'GPE': '', 'Job_Title': ''}
EDUCATION
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'N'}

mupdf: cannot open ../CFDS/110419126.doc: No such file or directory


Curriculum Vitae
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Curriculum Vitae'}
KENNEN S. GROSS, PhD, MPH
 DEGREE INFO  {'GPE': '', 'ORG': '', 'Bachelor': '', 'Master': 'MPH', 'Doctor': 'PhD', 'Major': 'KENNEN S GROSS', 'DATE': ''}
s  , NJ 08534
, NJ 08534
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': 'Pennington', 'Job_Title': ''}
SUMMARY OF QUALIFICATIONS
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Y'}
PROFESSIONAL EXPERIENCE
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': ''}
s  Chief Data Scientist - The Chartis Group Chicago, IL
Chief Data Scientist - , IL
 JOB INFO  {'DATE': '', 'ORG': ' : The Chartis Group Chicago', 'GPE': '', 'Job_Title': 'Chief Data Scientist'}
s  2013-2017
 JOB INFO  {'DATE': '2013-2017', 'ORG': '', 'GPE': '', 'Job_Title': ''}
s  Founder & President - Quantitative Innovations, , NJ
Founder & President - Quantitative Innovations, , NJ
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': 'Pennington', 'Job_Title': 'Founder  President

 DEGREE INFO  {'GPE': '', 'ORG': ' | Auburn University', 'Bachelor': '', 'Master': '', 'Doctor': 'Doctor of Philosophy', 'Major': 'Mathematics  – Auburn AL', 'DATE': ''}
s  2018
 JOB INFO  {'DATE': '2018', 'ORG': '', 'GPE': '', 'Job_Title': ''}
Master of Science, Applied Mathematics, National Chiao-Tung University – Hsin-Chu, Taiwan
 DEGREE INFO  {'GPE': 'Taiwan', 'ORG': ' | National Chiao-Tung University', 'Bachelor': '', 'Master': 'Master of Science', 'Doctor': '', 'Major': 'Chu', 'DATE': ''}
s  2009
 JOB INFO  {'DATE': '2009', 'ORG': '', 'GPE': '', 'Job_Title': ''}
Bachelor of Science, Applied Mathematics, National Chiao-Tung University – Hsin-Chu, Taiwan
 DEGREE INFO  {'GPE': 'Taiwan', 'ORG': ' | Chiao-Tung University', 'Bachelor': 'Bachelor of Science', 'Master': '', 'Doctor': '', 'Major': 'Chu', 'DATE': ''}
s  2007
 JOB INFO  {'DATE': '2007', 'ORG': '', 'GPE': '', 'Job_Title': ''}
PROFESSIONAL EXPERIENCE
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': ''}
s  Nov 2018 –

In [1018]:
all_job_titles.to_csv("30_CFDS_Resume_Headings.csv")

In [1005]:
resume_df

,Resume_Name,Block_Pos,Block_Title
0,82103515.pdf,0,Rohit Wason
1,82103515.pdf,5,RECENT EXPERIENCE
2,82103515.pdf,7,Science Architect
3,82103515.pdf,8,2018 - Present
4,82103515.pdf,11,Guide
5,82103515.pdf,13,Engineer/Architect
6,82103515.pdf,14,2014 - Present
7,82103515.pdf,22,Enterprise Architect
8,82103515.pdf,23,2011 - 2014
9,82103515.pdf,27,2010 - 2011


In [901]:

for newcol in ['DATE','ORG','GPE',"Bachelor","Master","Doctor","Job_Title","Type"]:
    resume_df[newcol]= ""
resume_df = resume_df.reset_index() 

In [615]:
tmp = resume_df.loc[start_t:end_extract_pos-1,:]

In [312]:
found_all= False

start_pos =  next(iter(resume_df[resume_df['Block_Pos']==start_t-1].index), -1)
continous_titles = pd.DataFrame()
continous = True
start_block = start_pos+1
end_block = start_pos+1
'''while continous:
    if (resume_df.Block_Pos[end_block] - resume_df.Block_Pos[end_block-1]) == 1:
        start_pos += 1
    else:
        continous = False'''
print(start_block, end_block)
#get_date_or_company("UNIVERSITY OF COLORADO BOULDER |GRADUATE COURSE")
#get_Title_Co_Commas("UNIVERSITY OF COLORADO BOULDER |GRADUATE COURSE")

10 10


In [264]:
tmp = resume_df.loc[start_t:end_extract_pos,:]
check_number_uppercases_words(resume_df.Block_Title[13])
if re.search("\A([A-Z][a-z]{3,}\s*){2,3}\.",resume_df.Block_Title[13]):
    print("found")

found


In [334]:
#resume_df = resume_df.reset_index()
start_pos =  next(iter(resume_df[resume_df['Block_Pos']==start_t-1].index), -1)
end_pos = next(iter(resume_df[resume_df['Block_Pos']==end_extract_pos-1].index), -1)
if not end_pos:
    end_pos,_ = resume_df.shape


    
    

9

In [ ]:
start_pos = start_t-1
for i in start_pos:end_extract_pos:
        

In [314]:
found_all= False
start_pos =  next(iter(resume_df[resume_df['Block_Pos']==start_t-1].index), -1)
continous_titles = pd.DataFrame()
continous = True
start_block = start_pos
end_block = start_pos
while continous:
    if (resume_df.Block_Pos[end_block] - resume_df.Block_Pos[end_block-1]) == 1:
        start_pos += 1
    else:
        continous = False

st = resume_df.loc[start_pos,"Block_Title"]
tp = get_Title_Co_Commas(st)
counter = 0
missing_val = []
for key, value in tp.items():
    if value != "":
        counter += 1
    else:
        missing_val.append(key)
if counter == 2:
    start_pos += 1
    if (resume_df.Block_Pos[start_pos] - resume_df.Block_Pos[start_pos-1]) == 1:
        st = resume_df.loc[start_pos,"Block_Title"]
        if missing_val[0] == "Dates":
            missing_date = "True"
            date_s = get_date(s)
            if date_s:
                tp["Dates"] = date_s
            else:
                start_pos +=1 
                if (resume_df.Block_Pos[start_pos] - resume_df.Block_Pos[start_pos-1]) == 1:
                        date_s = get_date(s)
                        tp["Dates"] = date_s
        else:
            tp[missing_val[0]] = st
elif counter == 1:
    print("Do nothing")

KeyError: 8

In [430]:
def add_flags_2_resume_df(resume_df):
    resume_df2 = resume_df.copy()
    
    for index, row in resume_df2.iterrows():
        _,_,job_t = list(get_job_title(row["Block_Title"]))[0]
        date_d, company_s = get_date_or_company(row["Block_Title"])
        _,_,date_s = get_date(row.Block_Title)
        city_s = extract_city(row["Block_Title"])
        degree_level = extract_degree_level(row["Block_Title"])
        if job_t:
            resume_df2.loc[index,"Job_Title"] = 1
            resume_df2.loc[index,"Job_Title_txt"] = job_t
            print("title :", job_t,row["Block_Pos"])
        if company_s:
            resume_df2.loc[index,"Company_Name"] = 1
            resume_df2.loc[index,"Company_Name_txt"] = company_s
            print("Company: ",company_s, row.Block_Pos)
        if date_s:
            resume_df2.loc[index,"Date_s"] = 1
            resume_df2.loc[index,"Date_s_txt"] = date_s
            print("Date: ",date_s,row.Block_Pos)
        if city_s:
            resume_df2.loc[index,"City_Name"] = 1
            resume_df2.loc[index,"City_Name_txt"] = city_s
        if degree_level:
            resume_df2.loc[index,"Degree_level"] = 1
            resume_df2.loc[index,"Degree_level_txt"] = degree_level
        if date_d and not date_s:
            resume_df2.loc[index,"Degree_Date"] = 1
            resume_df2.loc[index,"Degree_Date_txt"] = date_d
    return resume_df2

In [903]:
resume_df.dtypes

index           int64
Resume_Name    object
Block_Pos       int64
Block_Title    object
DATE           object
ORG            object
GPE            object
Bachelor       object
Master         object
Doctor         object
Job_Title      object
Type           object
dtype: object

In [997]:
def tag_headings(resume_df):
    for index, row in resume_df.iterrows():
        degree,degree_type = extract_degree_level(row["Block_Title"])
        if degree:
            
            degree_info = extract_all_info_degree(row["Block_Title"])
            #
            #    print(key, " * ",index)
            for key, val in degree_info.items():
                resume_df.loc[index,key] = val
        else:
            job_info = get_job_info(row["Block_Title"])
            print(" JOB INFO ", job_info)
            for key, val in job_info.items():
                resume_df.loc[index,key] = val
            #tp = [1 for x in list(job_info.values()) if x]
            #if tp:
             #   for key, val in job_info.items():
             #       print(key, " * ",index)
             #       resume_df.iloc[index,key] = val
    return resume_df

In [1000]:
start_t

29

In [1009]:
resume_df2 = tag_headings(resume_df)
resume_df2.loc[resume_df2.Block_Pos < start_t, "Job_Title"] = ""

MICHAEL LAWRENCE SMITH, M.S.
 DEGREE INFO  {'GPE': '', 'ORG': '', 'Bachelor': '', 'Master': 'M.S.', 'Doctor': '', 'Major': 'MICHAEL LAWRENCE SMITH', 'DATE': ''}
s  , Kentucky 40245
, Kentucky 40245
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': 'Louisville', 'Job_Title': 'Kentucky'}
s  Phone: 502-432-1542
Phone: 502-432-1542
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': 'Phone'}
WORK EXPERIENCE
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': '', 'Job_Title': ''}
s  QC Analytical Team Lead                                                                                     November 2016-Current
s  November 2016-Current
 JOB INFO  {'DATE': 'November 2016-Current', 'ORG': ' : QC Analytical Team Lead                                                                                     ', 'GPE': '', 'Job_Title': ''}
s  Creosalus                                                                                                                         , Kentucky
s  Creosalus                      

s  Las Vegas, 
Las Vegas,
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': 'Nevada', 'Job_Title': 'Las Vegas'}
s  Michael L. Smith, Pallab Pahari, Jurgen Rohr                                                                ACS National Meeting
s  Michael L. Smith, Pallab Pahari, Jurgen Rohr                                                                ACS National Meeting
s  Michael L. Smith, , Jurgen Rohr                                                                ACS National Meeting
s  Michael L. Smith, , Jurgen Rohr                                                                ACS National Meeting
Michael L. Smith, , Jurgen Rohr                                                                ACS National Meeting
 JOB INFO  {'DATE': '', 'ORG': ' : Pallab Pahari', 'GPE': '', 'Job_Title': 'Michael L Smith  Jurgen Rohr                                                                S National Meeting'}
s  San Francisco, 
San Francisco,
 JOB INFO  {'DATE': '', 'ORG': '', 'GPE': 'California',

In [680]:
resume_df2.to_csv("CheckITOUT.csv")

In [696]:
resume_df.reset_index(inplace=True, drop=True)
for index, row in resume_df.iterrows():
    print(resume_df.loc[index,"Block_Title"])

Manasa Reddy
Big Data Engineer
Work Experience
Big Data Engineer
SC Engineering Works
November 2015 to November 2016
Education
Master of Science in Technology
Jawaharlal Nehru Technological University
Skills
Additional Information
TECHNICAL SKILLS
Big Data Ecosystem: Hadoop, HDFS, YARN, MapReduce, Hive,
Pig, Spark, Sqoop, Tez
Languages: Python, SQL, Java, HTML,CSS.
Databases: MS Access, SQL Server
Editors: Eclipse
Operating Systems: Windows, Unix
Data Visualization: Tableau


In [29]:
#extract_degree_level("Ph.D., 2008")
#get_date("Ph.D., 2008")
extract_all_info_degree("University of Minnesota Sept. 2012 – Aug. 2016")

pattern1
University of Minnesota Sept. 2012 – Aug. 2016


Exception: url or text is required

In [482]:
extract_degree_level("Bachelors of Arts, Political Science - University of Minnesota")

''

In [431]:
resume_df2 = add_flags_2_resume_df(resume_df)

title : Software Engineer 1
title : Software Engineer 10
Company:  HeavyWater Inc -   11
Date:  2017-03 - P 11
title : Learning Engineer 17
Company:  Lumidatum Inc - 18
Date:  2016-11 - 2017-02 18
Company:  Master's in Computer Science 42
Company:  University of Texas 43
title : Computer Engineer 45
Company:  Computer Engineering 45
Company:  University of Pune -  Pune 46
Company:  Python 49
Company:  AWS 59
Company:  Ubuntu 60


In [419]:
#def get_start_end_continous_Rows(resume_df,)
found_block = False

resume_df2 = add_flags_2_resume_df(resume_df.query("index > "+str(start_pos)+" and index < "+str(end_extract_pos)))
resume_df2.reset_index(drop=True, inplace=True)

resume_df2["Sum_Experience_Val"] = resume_df2[["Job_Title","Company_Name","Date_s","City_Name"]].sum(axis = 1)
end_resume_df = False
index = 0
start = 0
len_resume_df2,_ = resume_df2.shape
start_end_df = pd.DataFrame()
for index, row in enumerate(resume_df2):
    if index+1 < len_resume_df2: 
        diff_val = resume_df2.Block_Pos[index+1] - resume_df2.Block_Pos[index]
        if resume_df2.Sum_Val[index] == 0:
            start_end_df = start_end_df.append({"start":resume_df2.Block_Pos[start],"end":resume_df2.Block_Pos[index],
                                               "sum_val":resume_df2.Sum_Experience_Val[start:index+1].sum(axis=0)}, ignore_index = True)
            start = index+1
        else:
            if  diff_val > 2:
                start_end_df = start_end_df.append({"start":resume_df2.Block_Pos[start],"end":resume_df2.Block_Pos[index],
                                                   "sum_val":resume_df2.Sum_Experience_Val[start:index+1].sum(axis=0)},ignore_index = True)
                start = index+1
            
        
        
    

/Users/dalila.benachenhou/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/dalila.benachenhou/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/dalila.benachenhou/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

title : Software Engineer 10


/Users/dalila.benachenhou/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Company:  HeavyWater Inc -   11


/Users/dalila.benachenhou/anaconda3/envs/allennlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Date:  2017-03 - P 11
title : Learning Engineer 17
Company:  Lumidatum Inc - 18
Date:  2016-11 - 2017-02 18


In [420]:
start_end_df

,end,start,sum_val
0,9.0,9.0,0.0
1,11.0,10.0,3.0
2,18.0,17.0,3.0
3,23.0,23.0,0.0


In [421]:
#start_end_df = start_end_df[1:,:]
finished = False
index = 0 
len_df,_ = start_end_df.shape
experience_df = pd.DataFrame()
counter = 1 
for idx in range(0,len_df-1):
    if start_end_df.sum_val[idx] > 0:
        text_data = ""
        for i in range(int(start_end_df.end[idx]+1),int(start_end_df.start[idx+1])):
            text_data = text_data+"\n"+resume_l[i]
        counter += 1
        experience_df = experience_df.append({"Experiences":text_data},ignore_index = True)
        if (counter > 3):
            break
experience_df

,Experiences
0,\n• Responsible for automation of services pro...
1,\nBuilding and automation of storing data (gig...


In [374]:
resume_df2 = resume_df2.assign(Sum_Val = 0)
resume_df2["Sum_Val"] = resume_df2[["Job_Title","Company_Name","Date_s","City_Name"]].sum(axis = 1)

In [407]:
start_end_df

,end,start,sum_val
0,9.0,9.0,0.0
1,11.0,10.0,3.0
2,18.0,17.0,3.0
3,23.0,23.0,0.0


In [235]:
i = resume_df[resume_df.Block_Title == "Forms and Interfaces."].index


Index(['Resume_Name', 'Block_Pos', 'Block_Title'], dtype='object')

In [243]:
resume_df2.loc[[47]]
resume_df2.query('index < 46')

,index,Resume_Name,Block_Pos,Block_Title,Job_Title,Company_Name,Date_s,City_Name
0,0,112712805.pdf,0,Suma Dodmani,0,0,0,0
1,3,112712805.pdf,3,EDUCATION,0,0,0,0
2,4,112712805.pdf,4,UNIVERSITY OF COLORADO,0,1,0,0
3,5,112712805.pdf,5,BOULDER,0,0,0,0
4,6,112712805.pdf,6,MS IN COMPUTER SCIENCE,0,0,0,0
5,7,112712805.pdf,7,SPECIALIZATION: DATA SCIENCE AND,0,0,0,0
6,8,112712805.pdf,8,ENGINEERING,0,0,0,0
7,9,112712805.pdf,9,"Graduated May 2019 Boulder, CO",0,0,0,0
8,10,112712805.pdf,10,BVBCET,0,1,0,0
9,11,112712805.pdf,11,BE IN INFORMATION SCIENCE,0,0,0,0


In [486]:
resume_df = resume_df.reset_index()
resume_df.shape

(97, 8)